In [5]:
import os
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [6]:
device = 'cuda'

In [28]:
class MyNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512), 
            nn.ReLU(), 
            nn.Linear(512, 10)
        )

        print(self.flatten)

    def forward(self, x):
        
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)

        return logits
        
    
        

In [29]:
model = MyNeuralNetwork()
model.to(device)

Flatten(start_dim=1, end_dim=-1)


MyNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [31]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


In [34]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab
y_pred = pred_probab.argmax(1)
y_pred

tensor([2], device='cuda:0')

In [35]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [41]:
flatten_image = nn.Flatten()(input_image)
flatten_image.size()

torch.Size([3, 784])

In [49]:
layer = nn.Linear(28 * 28, 5)

In [50]:
hidden= layer(flatten_image)
hidden.shape

torch.Size([3, 5])

In [51]:
print('before relu', hidden)
print('after relu', nn.ReLU()(hidden))

before relu tensor([[ 0.5604, -0.0643,  0.3646, -0.2663, -0.2583],
        [ 0.4041, -0.5093,  0.4110, -0.3681, -0.0205],
        [ 0.0308, -0.2447,  0.2197, -0.0534, -0.2499]],
       grad_fn=<AddmmBackward0>)
after relu tensor([[0.5604, 0.0000, 0.3646, 0.0000, 0.0000],
        [0.4041, 0.0000, 0.4110, 0.0000, 0.0000],
        [0.0308, 0.0000, 0.2197, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)


In [62]:
input = torch.ones(5)

w = torch.randn((5, 3), requires_grad=True)
b = torch.randn(3, requires_grad=True)

y = torch.zeros(3)

z = torch.matmul(input, w) + b

loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)



In [63]:
input, y, w, b

(tensor([1., 1., 1., 1., 1.]),
 tensor([0., 0., 0.]),
 tensor([[-1.2571, -0.9681,  0.4108],
         [-0.2173, -0.9500,  0.1241],
         [ 1.0952, -0.9420,  0.0205],
         [-0.5482,  0.2421,  1.0582],
         [-0.1975,  0.3992,  0.0865]], requires_grad=True),
 tensor([ 1.1791, -0.0636,  0.8721], requires_grad=True))

In [64]:
z.grad_fn

In [65]:
loss.grad_fn

In [68]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [70]:
print(w.grad)
print(b.grad)

tensor([[0.1712, 0.0309, 0.3097],
        [0.1712, 0.0309, 0.3097],
        [0.1712, 0.0309, 0.3097],
        [0.1712, 0.0309, 0.3097],
        [0.1712, 0.0309, 0.3097]])
tensor([0.1712, 0.0309, 0.3097])


In [71]:
z.requires_grad

True